In [1]:
import pandas as pd
import csv
import json

In [16]:
nwards = 24

masterfile = "Mumbai_master.xlsx"

df = pd.read_excel(masterfile,sheet_name="Demographics")
df = df.head(nwards)
df

wardLetterToNo = {}

wardNoToLetter = {}
for index, row in df.iterrows():
    wardLetterToNo[row["Alphabet"]] = row["wardNo"]
    wardNoToLetter[row["wardNo"]] = row["Alphabet"]
    
    wardLetterToNo[row["Alphabet"]+"_hd"] = row["wardNo"]+nwards
    wardNoToLetter[row["wardNo"]+nwards] = row["Alphabet"]+"_hd"

{1: 'A',
 25: 'A_hd',
 2: 'B',
 26: 'B_hd',
 3: 'C',
 27: 'C_hd',
 4: 'D',
 28: 'D_hd',
 5: 'E',
 29: 'E_hd',
 6: 'F/N',
 30: 'F/N_hd',
 7: 'F/S',
 31: 'F/S_hd',
 8: 'G/N',
 32: 'G/N_hd',
 9: 'G/S',
 33: 'G/S_hd',
 10: 'H/E',
 34: 'H/E_hd',
 11: 'H/W',
 35: 'H/W_hd',
 12: 'K/E',
 36: 'K/E_hd',
 13: 'K/W',
 37: 'K/W_hd',
 14: 'L',
 38: 'L_hd',
 15: 'M/E',
 39: 'M/E_hd',
 16: 'M/W',
 40: 'M/W_hd',
 17: 'N',
 41: 'N_hd',
 18: 'P/N',
 42: 'P/N_hd',
 19: 'P/S',
 43: 'P/S_hd',
 20: 'R/C',
 44: 'R/C_hd',
 21: 'R/N',
 45: 'R/N_hd',
 22: 'R/S',
 46: 'R/S_hd',
 23: 'S',
 47: 'S_hd',
 24: 'T',
 48: 'T_hd'}

In [24]:
def checkRowOrdering(df, n, name="(unknown-df)"):
    assert df.shape[0]==n, f"{name} does not have {n} rows"
    count = 0
    for index,row in df.iterrows():
        assert row["wardIndex"] == count, f"{name} has wardIndex mismatch in row {count}"
        count+=1

In [27]:
demographics_nonhd = df[["wardIndex", "wardNo", "Alphabet", "wardName", "area.sq.km.","nonslumDwePop"]]
demographics_nonhd = demographics_nonhd.sort_values('wardNo')
demographics_nonhd = demographics_nonhd.rename(columns={"nonslumDwePop": "totalPopulation"})

demographics_hd = df[["wardIndex", "wardNo", "Alphabet", "wardName", "area.sq.km.","slumDwePop"]]
demographics_hd = demographics_hd.rename(columns={"slumDwePop": "totalPopulation"})
demographics_hd = demographics_hd.sort_values('wardNo')

demographics_hd["wardNo"] = demographics_hd["wardNo"] + 24
demographics_hd["wardIndex"] = demographics_hd["wardIndex"] + 24
demographics_hd["Alphabet"] = demographics_hd["Alphabet"] + "_hd"
demographics_hd["wardName"] = demographics_hd["wardName"] + "_hd"

demographics_nonhd["hd_flag"] = 0
demographics_hd["hd_flag"] = 1


demographics = pd.concat([demographics_nonhd, demographics_hd], axis=0)
demographics['totalPopulation'] = demographics['totalPopulation'].astype(int)

checkRowOrdering(demographics, 48, "demographics")

#demographics


In [28]:
households_nonhd = df[["wardIndex", "wardNo", "Alphabet", "wardName", "Households"]]
households_nonhd = households_nonhd.sort_values('wardNo')
households_nonhd["Households"] = 0

households_hd = households_nonhd.copy()

households_hd["wardNo"] = households_hd["wardNo"] + 24
households_hd["wardIndex"] = households_hd["wardIndex"] + 24
households_hd["Alphabet"] = households_hd["Alphabet"] + "_hd"
households_hd["wardName"] = households_hd["wardName"] + "_hd"

households_nonhd["hd_flag"] = 0
households_hd["hd_flag"] = 1


households = pd.concat([households_nonhd,households_hd], axis=0)

checkRowOrdering(households, 48, "households")


#households


In [29]:
employment_nonhd = df[["wardIndex", "wardNo", "Alphabet", "wardName", "nonslumDwePop", "#Totalnonslumworkers"]]
employment_nonhd = employment_nonhd.rename(columns={"nonslumDwePop": "totalPopulation","#Totalnonslumworkers":"Employed" })


employment_hd = df[["wardIndex", "wardNo", "Alphabet", "wardName", "slumDwePop", "#TotalSlumworkers"]]
employment_hd = employment_hd.rename(columns={"slumDwePop": "totalPopulation","#TotalSlumworkers":"Employed" })
employment_hd["wardNo"] = employment_hd["wardNo"] + 24
employment_hd["wardIndex"] = employment_hd["wardIndex"] + 24
employment_hd["Alphabet"] = employment_hd["Alphabet"] + "_hd"
employment_hd["wardName"] = employment_hd["wardName"] + "_hd"


employment_nonhd["hd_flag"] = 0
employment_hd["hd_flag"] = 1


employment = pd.concat([employment_nonhd, employment_hd], axis=0)
employment["totalPopulation"] = employment["totalPopulation"].astype(int)
employment["Employed"] = employment["Employed"].astype(int)
employment["Unemployed"] = employment["totalPopulation"] - employment["Employed"]

checkRowOrdering(employment, 48, "employment")

In [70]:
OD_nonhd = pd.read_excel(masterfile,sheet_name="Inter-Ward Travel Data")
OD_nonhd = OD_nonhd.drop([0])
OD_nonhd = OD_nonhd.drop(columns=['Dest (area)'])
OD_nonhd= OD_nonhd.rename(columns={'Unnamed: 0':'wardNo'})
OD_nonhd["wardNo"] = OD_nonhd["wardNo"].apply(lambda x: wardLetterToNo[x])

cols = [a for a in OD_nonhd.columns if a != "wardNo"]
for col in cols:
    OD_nonhd[f"{col}_hd"] = 0
OD_hd = OD_nonhd.copy()
OD_hd["wardNo"] = OD_hd["wardNo"] + 24

OD = pd.concat([OD_nonhd, OD_hd], axis=0)
cols= [a for a in OD.columns if a != "wardNo"]

OD = OD.rename(columns={a: wardLetterToNo[a] for a in cols})
OD = OD.sort_values('wardNo')

cols = [a for a in OD.columns if a != 'wardNo']
OD = OD[["wardNo"] + sorted(cols)]

,wardNo,1,2,3,4,5,6,7,8,9,...,39,40,41,42,43,44,45,46,47,48
1,1,0.340872,0.068208,0.049056,0.180012,0.201852,0.016920,0.018222,0.013014,0.011844,...,0,0,0,0,0,0,0,0,0,0
2,2,0.340872,0.068208,0.049056,0.180012,0.201852,0.016920,0.018222,0.013014,0.011844,...,0,0,0,0,0,0,0,0,0,0
3,3,0.340872,0.068208,0.049056,0.180012,0.201852,0.016920,0.018222,0.013014,0.011844,...,0,0,0,0,0,0,0,0,0,0
4,4,0.340872,0.068208,0.049056,0.180012,0.201852,0.016920,0.018222,0.013014,0.011844,...,0,0,0,0,0,0,0,0,0,0
5,5,0.340872,0.068208,0.049056,0.180012,0.201852,0.016920,0.018222,0.013014,0.011844,...,0,0,0,0,0,0,0,0,0,0
6,6,0.089276,0.017864,0.012848,0.047146,0.052866,0.186120,0.200442,0.143154,0.130284,...,0,0,0,0,0,0,0,0,0,0
7,7,0.089276,0.017864,0.012848,0.047146,0.052866,0.186120,0.200442,0.143154,0.130284,...,0,0,0,0,0,0,0,0,0,0
8,8,0.089276,0.017864,0.012848,0.047146,0.052866,0.186120,0.200442,0.143154,0.130284,...,0,0,0,0,0,0,0,0,0,0
9,9,0.089276,0.017864,0.012848,0.047146,0.052866,0.186120,0.200442,0.143154,0.130284,...,0,0,0,0,0,0,0,0,0,0
10,10,0.028406,0.005684,0.004088,0.015001,0.016821,0.014100,0.015185,0.010845,0.009870,...,0,0,0,0,0,0,0,0,0,0


In [71]:
demographics.to_csv('demographics.csv', index=False)
employment.to_csv('employment.csv',index=False)
households.to_csv('households.csv', index=False)
OD.to_csv('ODMatrix.csv', index=False)